In [5]:
!pip install keras
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

  Using cached tensorflow-1.2.1-cp35-cp35m-win_amd64.whl
Requirement already up-to-date: six>=1.10.0 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: backports.weakref==1.0rc1 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: wheel>=0.26 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: werkzeug>=0.11.10 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: protobuf>=3.2.0 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: numpy>=1.11.0 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-date: bleach==1.5.0 in c:\users\java\anaconda3\envs\tensorflow-gpu\lib\site-packages (from tensorflow)
Requirement already up-to-

In [25]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

In [26]:
import tensorflow as tf

In [14]:
tfconfig = tf.ConfigProto(log_device_placement = True) 
# # 使用tf.ConfigProto建立session的組態設定tfconfig，參數log_device_placement設定為True，代表要顯示裝置相關資訊
# with tf.session(config = tfconfig) as sess: # 開啟session時，傳入之前建立的session的組態tfconfig

In [15]:
# 未來可以畫圖計算cpu vs gpu p336

In [32]:
###############################
#                             #
#          DEMO_07            #  上面已經安裝gpu，嘗試使用確認是否有加快速度
#                             #
###############################

# 解bug
# Python：IOError: image file is truncated 的解决办法
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # 解bug

#############################################################################################

try:
    train_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/train'
    validation_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/validation'
    nb_train_samples = 11871
    nb_validation_samples = 1800
    epochs = 15 # 迭代
    batch_size = 100

    ##################################################################

    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(28, 28),
            batch_size=100,
            class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(28, 28),
            batch_size=100,
            class_mode='categorical')

    ##################################################################

    print(train_generator.class_indices)  # 印出標籤
    print(validation_generator.class_indices)  # 印出標籤

    ##################################################################

    model = Sequential() # 建立model

    ##################################################################

    # 加入卷積層1
    model.add(Conv2D(filters=32, kernel_size = (3,3),   # 32個3*3濾鏡
                     input_shape = (28,28,3),   # 圖片大小&三原色
                     activation = 'relu',
                     padding = 'same' # 經過此層影像大小不變
                     ) 
             )


    # 控制overfit
    model.add(Dropout(rate=0.25))    


    # 加入池化層1
    model.add(MaxPooling2D(pool_size=(2,2))) 

    ##################################################################

    # 加入卷積層2
    model.add(Conv2D(filters=64, kernel_size = (3,3),   # 64個3*3濾鏡
                     activation = 'relu',
                     padding = 'same' # 經過此層影像大小不變
                     ) 
             )


    # 控制overfit
    model.add(Dropout(rate=0.25))          



    # 加入池化層2
    model.add(MaxPooling2D(pool_size=(2,2))) 

    ##################################################################

    # 加入平坦層
    model.add(Flatten()) #28*28*64


    # 控制overfit
    model.add(Dropout(rate=0.25))    


    ##################################################################

    # 加入隱藏層
    model.add(Dense(16384, activation = 'relu'))


    # 控制overfit
    model.add(Dropout(rate=0.25))    

    ##################################################################

    # 建立輸出層
    model.add(Dense(9, activation = 'softmax'))

    ##################################################################

    # 查看模型摘要
    print(model.summary())

    ##################################################################

    ###############################
    #                             #
    #          準備訓練            #
    #                             #
    ###############################
    model.compile(loss = 'categorical_crossentropy',  # 損失函數
                  optimizer = 'adam',                 # 最優化方法
                  metrics = ['accuracy'])             # 以準確率評估

##################################################################

    train_history = model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps= nb_validation_samples // batch_size)

    show_train_history(train_history,'acc','val_acc')
except:
    pass
#########################################################################################

# 結論，只快了一點點，效果不明顯                                                                  
                                                                                    
#########################################################################################


Found 11871 images belonging to 9 classes.
Found 1800 images belonging to 9 classes.
{'京都塔': 2, '平等院鳳凰堂': 5, '二條城': 1, '京都御所': 3, '三十三間堂': 0, '金閣寺': 8, '渡月橋': 7, '千本鳥居': 4, '清水寺': 6}
{'京都塔': 2, '平等院鳳凰堂': 5, '二條城': 1, '京都御所': 3, '三十三間堂': 0, '金閣寺': 8, '渡月橋': 7, '千本鳥居': 4, '清水寺': 6}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 14, 14, 32)        896       
_________________________________________________________________
dropout_93 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 7, 7, 32)          9248      
_________________________________________________________________
dropout_94 (Dropout)         (None, 7, 7, 32)          0   

ResourceExhaustedError: OOM when allocating tensor of shape [3,3,32,32] and type float
	 [[Node: Const_882 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,3,32,32] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Const_882', defined at:
  File "C:\Users\Java\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Java\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Java\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Java\Anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Java\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Jav

In [9]:
###############################
#                             #
#          DEMO_08            #  只測試三輪，確認是否有使用gpu
#                             #
###############################


# 解bug
# Python：IOError: image file is truncated 的解决办法
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # 解bug

#############################################################################################

def imager():
    try:
        train_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/train'
        validation_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/validation'
        nb_train_samples = 11871
        nb_validation_samples = 1800
        epochs = 3 # 迭代
        batch_size = 100

        ##################################################################

        train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)

        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
                train_data_dir,
                target_size=(28, 28),
                batch_size=100,
                class_mode='categorical')

        validation_generator = test_datagen.flow_from_directory(
                validation_data_dir,
                target_size=(28, 28),
                batch_size=100,
                class_mode='categorical')

        ##################################################################

        print(train_generator.class_indices)  # 印出標籤
        print(validation_generator.class_indices)  # 印出標籤

        ##################################################################

        model = Sequential() # 建立model

        ##################################################################

        # 加入卷積層1
        model.add(Conv2D(filters=32, kernel_size = (3,3),   # 32個3*3濾鏡
                         input_shape = (28,28,3),   # 圖片大小&三原色
                         activation = 'relu',
                         padding = 'same' # 經過此層影像大小不變
                         ) 
                 )


        # 控制overfit
        model.add(Dropout(rate=0.25))    


        # 加入池化層1
        model.add(MaxPooling2D(pool_size=(2,2))) 

        ##################################################################

        # 加入卷積層2
        model.add(Conv2D(filters=64, kernel_size = (3,3),   # 64個3*3濾鏡
                         activation = 'relu',
                         padding = 'same' # 經過此層影像大小不變
                         ) 
                 )


        # 控制overfit
        model.add(Dropout(rate=0.25))          



        # 加入池化層2
        model.add(MaxPooling2D(pool_size=(2,2))) 

        ##################################################################

        # 加入平坦層
        model.add(Flatten()) #28*28*64


        # 控制overfit
        model.add(Dropout(rate=0.25))    


        ##################################################################

        # 加入隱藏層
        model.add(Dense(16384, activation = 'relu'))


        # 控制overfit
        model.add(Dropout(rate=0.25))    

        ##################################################################

        # 建立輸出層
        model.add(Dense(9, activation = 'softmax'))

        ##################################################################

        # 查看模型摘要
        print(model.summary())

        ##################################################################

        ###############################
        #                             #
        #          準備訓練            #
        #                             #
        ###############################
        model.compile(loss = 'categorical_crossentropy',  # 損失函數
                      optimizer = 'adam',                 # 最優化方法
                      metrics = ['accuracy'])             # 以準確率評估

    ##################################################################
        with tf.device("/gpu:0"):
            train_history = model.fit_generator(
                                train_generator,
                                steps_per_epoch=nb_train_samples // batch_size,
                                epochs=20,
                                validation_data=validation_generator,
                                validation_steps= nb_validation_samples // batch_size)

            show_train_history(train_history,'acc','val_acc')
    except:
        pass

###############################################################

# tfconfig = tf.ConfigProto(log_device_placement = True) 
# # 使用tf.ConfigProto建立session的組態設定tfconfig，參數log_device_placement設定為True，代表要顯示裝置相關資訊
# with tf.Session(config = tfconfig) as sess: 
# # 開啟session時，傳入之前建立的session的組態tfconfig
#     result = sess.run(imager())

#########################################################################################

# 結論，黑人問號，明明只有用cpu

#########################################################################################

imager()

Found 11871 images belonging to 9 classes.
Found 1800 images belonging to 9 classes.
{'京都塔': 2, '平等院鳳凰堂': 5, '二條城': 1, '京都御所': 3, '三十三間堂': 0, '金閣寺': 8, '渡月橋': 7, '千本鳥居': 4, '清水寺': 6}
{'京都塔': 2, '平等院鳳凰堂': 5, '二條城': 1, '京都御所': 3, '三十三間堂': 0, '金閣寺': 8, '渡月橋': 7, '千本鳥居': 4, '清水寺': 6}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
dropout_17 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_18 (Dropout)         (None, 14, 14, 64)        0   

In [ ]:
###############################
#                             #
#          DEMO_09            #  只測試三輪，嘗試指定GPU
#                             #
###############################


# 解bug
# Python：IOError: image file is truncated 的解决办法
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # 解bug

#############################################################################################



train_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/train'
validation_data_dir = 'C:/Users/Java/Desktop/Project/Project_Image/data3/validation'
nb_train_samples = 11871
nb_validation_samples = 1800
epochs = 3 # 迭代
batch_size = 100

##################################################################

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(28, 28),
        batch_size=100,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(28, 28),
        batch_size=100,
        class_mode='categorical')

##################################################################

print(train_generator.class_indices)  # 印出標籤
print(validation_generator.class_indices)  # 印出標籤

##################################################################

model = Sequential() # 建立model

##################################################################

# 加入卷積層1
model.add(Conv2D(filters=32, kernel_size = (3,3),   # 32個3*3濾鏡
                 input_shape = (28,28,3),   # 圖片大小&三原色
                 activation = 'relu',
                 padding = 'same' # 經過此層影像大小不變
                 ) 
         )


# 控制overfit
model.add(Dropout(rate=0.25))    


# 加入池化層1
model.add(MaxPooling2D(pool_size=(2,2))) 

##################################################################

# 加入卷積層2
model.add(Conv2D(filters=64, kernel_size = (3,3),   # 64個3*3濾鏡
                 activation = 'relu',
                 padding = 'same' # 經過此層影像大小不變
                 ) 
         )


# 控制overfit
model.add(Dropout(rate=0.25))          



# 加入池化層2
model.add(MaxPooling2D(pool_size=(2,2))) 

##################################################################

# 加入平坦層
model.add(Flatten()) #28*28*64


# 控制overfit
model.add(Dropout(rate=0.25))    


##################################################################

# 加入隱藏層
model.add(Dense(16384, activation = 'relu'))


# 控制overfit
model.add(Dropout(rate=0.25))    

##################################################################

# 建立輸出層
model.add(Dense(9, activation = 'softmax'))

##################################################################

# 查看模型摘要
print(model.summary())

##################################################################

###############################
#                             #
#          準備訓練            #
#                             #
###############################
model.compile(loss = 'categorical_crossentropy',  # 損失函數
              optimizer = 'adam',                 # 最優化方法
              metrics = ['accuracy'])             # 以準確率評估

##################################################################
train_history = model.fit_generator(
                    train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps= nb_validation_samples // batch_size)


# ###############################################################
# tfconfig = tf.ConfigProto(log_device_placement = True) 
# # 使用tf.ConfigProto建立session的組態設定tfconfig，參數log_device_placement設定為True，代表要顯示裝置相關資訊
# with tf.Session(config = tfconfig) as sess: 
#     with tf.device("/gpu:0"):
#         imager()
#     sess.run(imager())

#     # 開啟session時，傳入之前建立的session的組態tfconfig


#########################################################################################

# 結論，黑人問號，明明只有用cpu

#########################################################################################



In [4]:
with tf.device("/gpu:0"):
    imager()

NameError: name 'imager' is not defined